El ejercicio cuanta tres parte que van por niveles creciente de dificultat, segon vuestro nivel de programacion es possible que no podais hacerlo integralmente (y no pasa nada). Lo importante es hacer bien lo que consigais. La primera parte es accessible para todo el mundo, toda la informacion necesaria esta en el notebook de la clase (y de las clases anteriores), y vale 6 puntos asi que es suficiente para aprobar.

Os aconsejo, poner os un limite de 2h para hacer este ejercicio (lo repito no hace falta hacer todo). Cada pregunta de la primera parte, tiene un tiempo maximo, si os pasais de este tiempo es que seguramente no habeis entendido algo fundamental, en este caso contactar conmigo.

Por las preguntas de la parte dos y tres puede ser que tengais que buscar informacion en google, y pensar bastante. Si quiereis pasar mas de dos horas porque os gusta, porque quereis aprender mas, o porque quereis tener un 10 mas todo, porsupesto sois libre de hacerlo...

# 0. Reading data (just run it)

In [2]:
import numpy as np
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from netCDF4 import Dataset, num2date, date2num
import numpy.ma as ma
import math

Here we define two auxiliary function that are used to averaged spatially the data

In [3]:
def extend_table(array, dims_expend):
    """function that extend the dimension of an array repeating it
    array: an array containing the variable to be repeated
    dims_expend: the dimension to be added"""
    array_ext=array
    dims_expend=list(dims_expend)
    dims_expend.reverse()
    for dim in dims_expend:
        array_ext=np.expand_dims(array_ext, axis=0).repeat(dim, axis=0)
        
    return(array_ext)

def area_av(array, pos_lat, pos_lon, lats, lons):
    """fonction computing the regional average
    array: an array containing the variable to be averaged spatially
    pos_lat: position of the latitude
    pos_lon: position of the longitude
    lats: latitude corresponding to the array
    lons: longitude corresponding to the area
    return: spatially averaged array"""

    # Compute the delta lon and delta lat (difference bteween two points)  
    dlon=list(lons[1:]-lons[:-1])
    dlon.append(lons[-2]-lons[-1])
    dlat=list(lats[1:]-lats[:-1])
    dlat.append(lats[-2]-lats[-1])

    #convert it in numpy array 
    dlon=np.array(dlon)%360 # apply modulo in case of difference >360
    dlat=np.array(dlat)

    
    #extend the lats, and cos over all the dimensions
    dim=array.shape   
    dlon_ext=np.swapaxes(extend_table(dlon, np.delete(dim, pos_lon)), len(dim)-1, pos_lon)
    dlat_ext=np.swapaxes(extend_table(dlat, np.delete(dim, pos_lat)), len(dim)-1, pos_lat)
    coslat_ext=np.swapaxes(extend_table(np.cos(lats*math.pi/180), np.delete(dim, pos_lat)), 
                           len(dim)-1, pos_lat)
                  
    #create a weighted array 
    weights=ma.array(dlon_ext*dlat_ext*coslat_ext, mask=array.mask)
    
    #make the weighted sum
    sumweigth=np.ma.sum(np.ma.sum(weights, axis=pos_lon),pos_lat) #also handled masked array
    array_av = np.ma.sum(np.ma.sum(weights*array, axis=pos_lon),pos_lat)/sumweigth

    array_av = np.ma.mean(np.ma.mean(array, axis=pos_lon),pos_lat)
    return(array_av)

We read data from a THREDDS or OPENDAB server, the advantge of the kind of data is that you can access directly the file without downloading it. You just download the subset of data you need (in the case precipitation over india)

http://apdrc.soest.hawaii.edu/

In [4]:
# define the box for india
lat_bnds = [5,25] 
lon_bnds = [70,90]
varname="precip" #sst
# url of the THREDDS servers were precipitation product are (GPCP 2.2)
url='http://apdrc.soest.hawaii.edu:80/dods/public_data/satellite_product/GPCP/monthly/v2.2_psg' 

#open ile
varf = Dataset(url)

#read longitude and latitude
lats = varf.variables['lat'][:] 
lons = varf.variables['lon'][:]

#select indices corresponding to the indian box
lat_inds = np.where((lats > lat_bnds[0]) & (lats < lat_bnds[1]))[0]
lon_inds = np.where((lons> lon_bnds[0]) & (lons < lon_bnds[1]))[0]
    
#read time axis
timevar=varf.variables["time"]
dates = num2date(timevar[:], units=timevar.units) #, calendar=timevar.calendar)


#download data subset
varobs=varf.variables[varname][:,lat_inds,lon_inds]

#close the file
varf.close()

#select corresponding latitude
lats_reg=lats[lat_inds]
lons_reg=lons[lon_inds]
        
# area average   
varobs = area_av(varobs, 1, 2, lats_reg, lons_reg)

print("We have now monthly precipitations over india from "+dates[0].strftime("%d/%m/%Y")+" to "+dates[-1].strftime("%d/%m/%Y"))
print("Stored in the variable varobs and the corresponding dates in the variable dates")

We have now monthly precipitations over india from 01/01/1979 to 01/10/2015
Stored in the variable varobs and the corresponding dates in the variable dates


Convertimos los numpy array dates y varobs en list, los que conoceis numpy y quereis hacer la practica utilizando numpy no ejecutais esta celda (mejor borrar la). Si veis que la question puede ser respondida por una funcion existente en numpy podeis responder con la funcion numpy. No hace falta definir una nueva funcion.

In [5]:
dates = [date(d.year, d.month, d.day) for d in list(dates)]
varobs = list(varobs)

# Para practicar (7pts)

1) Escribir una funcion que calcula la media de una lista, utilizar la para calcular la media de precipitaciones en India. No olvidar escribir la description de la funcion. Utilizar esta funcion para calcular la precipitation media en India.

2) Escribir una funcion que seleciona un año concreto dentro de un lista de fechas y devuelve los indices correspondientes.  No olvidar escribir la description de la funcion.

3) Escribir una funcion que seleciona unos indices dentro de una lista.  No olvidar escribir la description de la funcion. 

Ex:

select_ind([5,6,7,8],[1,3]) devuelve [6,8].

4) Utilizando las funciones de la question 2 y 3, verificar que las dos funciones hacen lo que pensais.

3) Utilizar las funciones de la question 1 et 2, 3 para calcular la precipitation media en 2010

4) Escribir una funcion que coge en argumento un año, la lista de valor de la precipitation y las fechas y devuelve True o False segon si huvo mas o menos lluvia que la media durante este año y imprime una frase tipo " el año ?? ha sido mas lluvioso que la media". Utilizar la por el año 2009.

# Mas dificil (3pts)

5) Escribir una funcion que selecciona los indices que coresponden a unos meses dentro de una lista de fechas.

6) Calcular la media de las precipitaciones en los meses de Junio, Julio, Augosto, y estimar el percentage de la precipitation occure en verano (Junio-Julio-Agosto)

7) Escribir una funcion que coge en argumento una lista de valor, la lista de fecha correspondientes, una lista de meses y calcula or cada año la media de precipitaciones que occuren durante estos meses

7) Calcular los 7 años mas y menos lluvioso

8bonus) Aixi a simple vista, pensais que hay un cambio en el monsoon relacionado 1) al cambio climatico 2) al fenomeno ENSO (ver la pagina web: https://ggweather.com/enso/oni.htm)

It's dificult to relate changes without a robust statistic analysis, but we can say that 5/7 of the smallest monsoon occurs after 2000, which might suggest that the monsoon is decreasing with climate change. 

About the relacion with El Niño, almost all year with a minum/maximum monsoon are occuring simultaneously with a positive/negative El Niño.